# Good Python plotting practices: B) aplpy

### ESOpy 2.0 @ Vitacura - 09-11 January 2018 - fvogt@eso.org


** BEFORE we start:**:
- MATPLOTLIB is *great* :  http://matplotlib.org/
- many tools wrap around it, e.g. *aplpy* for WCS and .fits files: https://aplpy.github.io/

**premise:** 
- matplotlib is the one-stop shop for most 2-D plotting in Python. To figure out how to do stuff -> online gallery:
http://matplotlib.org/gallery.html

- **But at times frustrating** to get beautiful & complex plots: 
   - subplots locations, sizes, ratios
   - colorbar location, size, width, alignment
   - labels outside plotting window 
   - layout updates 
   - fontsizes, ...
   - **display WCS coordinates from FITS header**

**aplpy:** 
- a wrapper module around matplotlib to deal with WCS data for you
- lots of nifty little bells and whistles, but still a wrapper -> all matplotlib ground layer accessible if needed
- https://aplpy.github.io/

**utility**: 
- very useful for plotting FITS files 

**cost**: 
- same as any matplotlib plots (matplotlib and aplpy are **not** exclusive to one another!)

**gain**:
- no need to understand the dark magic of WCS coordinates ...

### A) Set the stage: import the required modules

In [3]:
import os

# Use some notebook magic - forget the next line in normal scripted code
%pylab tk

import matplotlib.pyplot as plt # Gives access to basic plotting functions

import matplotlib.gridspec as gridspec # GRIDSPEC !
import aplpy


Populating the interactive namespace from numpy and matplotlib


/Users/fvogt/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


### B) List the path to some FITS files of interest

In [5]:
# A simple DSS2-R band image, obtained from http://archive.eso.org/dss/dss
fits_fn = os.path.join('..','data','NGC6121-off2-N-astrom_DSS2-Red.fits') 


### C) Get started with the plotting 

In [49]:
# First, create the figure
plt.close(1)
fig = plt.figure(1, figsize=(14.17,7)) # Please, someone asks me where 14.17 comes from !
plt.show()


### D) [Optional] Use gridspec to set the scene

In [50]:
# Now, create the gridspec structure, as required
gs = gridspec.GridSpec(1,2, height_ratios=[1], width_ratios=[1,1],
                       left=0.15, right=0.95, bottom=0.08, top=0.93, wspace=0.5, hspace=0.)

# 1 row, 2 columns, each with the required size ratios. 

fig.show() # This still shows a blank plots !


### E) Show the FITS files with aplpy

In [51]:
# For the first subplot, specify its location "manually" (no gridspec)
ax1 = aplpy.FITSFigure(fits_fn, figure=fig, north=False, 
                          subplot=[0.12,0.08,0.4,0.85])
ax1.show_grayscale(invert = True, stretch='arcsinh') # Show the image in a grayscale arcsinh stretch

# For the second plot, do the same, but this time using gridspec (for good measure)
# First, extract the gridspec ax bound values
gsb = list(gs[0,1].get_position(fig).bounds)
#print(gsb)

ax2 = aplpy.FITSFigure(fits_fn, figure=fig, north=False,
                          subplot=list(gsb))
ax2.show_colorscale(cmap='Oranges_r', stretch='linear',pmin=50,pmax=100,)


INFO: Auto-setting vmin to  4.095e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.469e+04 [aplpy.core]
INFO: Auto-setting vmin to  2.898e+03 [aplpy.core]
INFO: Auto-setting vmax to  3.132e+04 [aplpy.core]


### F) Some of the other aplpy features

In [52]:
# Add contours from (the same, or another) fits file
ax1.show_contour(fits_fn, levels=[1e4,2.4e4], smooth=3, colors='darkorange')

In [53]:
# Adding markers for position of interest
ax1.show_markers([245.9857],[-26.558259], marker='o',facecolor='None', s=1e3,edgecolor='darkorchid',zorder=10, lw=2)

In [54]:
# Adding a scalebar
for ax in [ax1,ax2]:
   ax.add_scalebar(60./3600,label='60" = 27 kpc', corner = 'bottom left', color = 'k', frame=1, linewidth=2)

#ax1.scalebar.show(scl, label=scll, corner = 'bottom left', color = 'k', frame=1)
#ax1.scalebar.set_linewidth(2)

/Users/fvogt/anaconda3/lib/python3.6/site-packages/aplpy/overlays.py:369: UserWarning: Text labels do not have attribute linewidth. Skipping.
  warnings.warn("Text labels do not have attribute {0}. Skipping.".format(kwarg))


In [55]:
# Zoom-in on a region of interest
ax2.recenter(245.9857,-26.558259, radius=120/3600)

### G) Fine tune the look of it (because we can)

In [56]:
for ax in [ax1,ax2]:
    ax.tick_labels.set_xformat('hh:mm:ss')
    ax.axis_labels.set_xpad(10)
    ax.ticks.set_linewidth(1.5)
    ax.set_tick_size(10)
    ax.set_tick_color('k')
    ax.axis_labels.set_xtext('R.A. [J2000]')

    ax.axis_labels.set_ytext('Dec. [J2000]')
    ax.axis_labels.set_ypad(-10)


### H) Exercise

Take a look at **exo_aplpy.py** ... can you fill the blanks ?

If you're stuck, look here: https://aplpy.readthedocs.io/en/v0.9.9/_generated/aplpy.aplpy.FITSFigure.html


In [7]:
# Complete the code below to turn this into a proper finding chart for the instrument of your choice
# First, create the figure
plt.close(2)
fig = plt.figure(2, figsize=(6.92,7))

# Let's keep things simple here ... only one plot
ax1 = aplpy.FITSFigure(fits_fn, figure=fig, north=False, 
                          subplot=[0.12,0.08,0.85,0.85])
ax1.show_grayscale(invert = True, stretch='arcsinh') # Show the image in a grayscale arcsinh stretch

# Some coordinates of interest
target = np.array([245.95570, -26.48259])
vlt_gs = np.array([246.03592, -26.41527])

# The field-of-view of your instrument, in degrees (center is at [0,0])
# (Bonus points to the first one to shout out loud which instrument this corresponds to ...)
inst_fov= np.array([[0.0086683878,   0.0084439565],[359.99128-360., 0.008499508],
                    [359.99172-360.,-0.0082782215],[0.0080579666,  -0.008389310]])

# ----------------------- Now it's your turn ! ----------------------------

# Step 1 (1 line): show the target location with a cross

# Step 2 (1 line): show the VLT guide star location

# Step 3 (1 line): recenter on the area of interest

# Step 4 (1 line): add a scalebar of 2 arcmin to the top right corner

# Step 5 (0 line): fix the plot so all the labels are visible

# Step 6 (5 lines): show the instrument field-of-view [ax.show_polygons() may help ...]

# -------------------------------------------------------------------------

plt.show()


INFO: Auto-setting vmin to  4.095e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.469e+04 [aplpy.core]


### I) And finally, some shameless advertisement for my project ... 

You choose what you want to focus on, I provide the rest:

- FITS / WCS plotting with aplpy
- automated queries to surveys (DSS2, GAIA)
- dealing with Sky Coordinates in astropy
- code structuring and packaging [intermediate-advanced]
- code documentation with sphinx [intermediate-advanced]
- interacting with p2 [intermediate-advanced]

![demo-image](./WFM_AO_starstuff_DSS2-Red.png)